In [190]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
from datetime import datetime, timedelta
import openpyxl 
import os
from pathlib import Path
import xlsxwriter
import send_email

from IPython.display import display, HTML

In [191]:
#Publisher Config
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
publisher_config = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
pub_config = publisher_config[0].get_as_df()
pub_config.rename(columns={'DP.DS or DP.sV': 'DP.SV'}, inplace=True)

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


In [192]:
sms_offer = infrastructure.get_smartsheet('offers_sms')
sms_offer = sms_offer[sms_offer['Hitpath Offer ID'].isna() == False]

In [193]:
file_name = 'sales-2024-07_exported_on_2024_07_15_11_39_23.csv'
file_path = os.path.join('input', file_name)

# Read the CSV file
df = pd.read_csv(file_path)

In [194]:
filtered_df = df[df['affiliate_name'].str.contains('SMS', na=False)]
filtered_df = filtered_df.dropna(subset=['campaign_id'])

In [195]:
sms_offer['Hitpath Offer ID'] = sms_offer['Hitpath Offer ID'].astype(int).astype(str)
filtered_df['campaign_id'] = filtered_df['campaign_id'].astype(int).astype(str)

sms_offer.rename(columns={'Hitpath Offer ID': 'hitpath_offer_id'}, inplace=True)
filtered_df.rename(columns={'campaign_id': 'hitpath_offer_id'}, inplace=True)

In [196]:
#merge Vertical by Hitpath Offer ID from sms_offer with filtered_df
filtered_df = pd.merge(filtered_df, sms_offer[['hitpath_offer_id', 'Vertical', 'Offer Name']], on='hitpath_offer_id', how='left')


In [197]:
#if campaign_name is NaN and hitpath_offer_id has associating Campaign Name, then fill the NaN with Campaign Name
filtered_df['campaign_name'] = np.where(filtered_df['campaign_name'].isna() & filtered_df['Offer Name'].notna(), filtered_df['Offer Name'], filtered_df['campaign_name'])

In [198]:
# Group by affiliate_name and calculate total amount
affiliate_totals = filtered_df.groupby('affiliate_name')['amount'].sum().reset_index()
top_affiliates = affiliate_totals.sort_values(by='amount', ascending=False).head(5)

# Group by campaign_id and calculate total amount
campaign_totals = filtered_df.groupby(['hitpath_offer_id', 'campaign_name'])['amount'].sum().reset_index()
top_campaigns = campaign_totals.sort_values(by='amount', ascending=False).head(5)

# Group by advertiser_name and calculate total amount
advertiser_totals = filtered_df.groupby('advertiser_name')['amount'].sum().reset_index()
top_advertisers = advertiser_totals.sort_values(by='amount', ascending=False).head(5)

# Group by Vertical and calculate total amount
vertical_totals = filtered_df.groupby('Vertical')['amount'].sum().reset_index()
top_verticals = vertical_totals.sort_values(by='amount', ascending=False).head(5)

# Start of the HTML email text
email_html = "<html><body>"

email_html += "<h1 style='color: black;'>Top Contributors</h1>"

# Top 5 Contributors by Affiliate Name
email_html += "<h2 style ='color:black;'>Top 5 Contributors by Affiliate Name:</h2><ol>"
for i, row in enumerate(top_affiliates.itertuples(), start=1):
    email_html += f"<li style ='color:black;'> {row.affiliate_name}: ${row.amount:.2f}</li>"
email_html += "</ol>"

# Top 5 Contributors by Campaign
email_html += "<h2 style ='color:black;'>Top 5 Contributors by Campaign:</h2><ol>"
for i, row in enumerate(top_campaigns.itertuples(), start=1):
    campaign_name = row.campaign_name if pd.notna(row.campaign_name) else "No Name"
    email_html += f"<li style ='color:black;'> {row.hitpath_offer_id} - {campaign_name}: ${row.amount:.2f}</li>"
email_html += "</ol>"

# Top 5 Contributors by Advertiser Name
email_html += "<h2 style ='color:black;'>Top 5 Contributors by Advertiser Name:</h2><ol>"
for i, row in enumerate(top_advertisers.itertuples(), start=1):
    email_html += f"<li style ='color:black;'> {row.advertiser_name}: ${row.amount:.2f}</li>"
email_html += "</ol>"

# Top 5 Contributors by Vertical
email_html += "<h2 style ='color:black;'>Top 5 Contributors by Vertical:</h2><ol>"
for i, row in enumerate(top_verticals.itertuples(), start=1):
    email_html += f"<li style ='color:black;'> {row.Vertical}: ${row.amount:.2f}</li>"
email_html += "</ol>"

email_html += "</body></html>"


display(HTML(email_html))
